모델 학습을 위한 데이터셋을 생성

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers &> /dev/null
from transformers import BertTokenizer
from transformers import AutoTokenizer

import csv
import gzip
import json
import pickle
import numpy as np

In [ ]:
def preprocess_amazon(json_filename = 'Books_5.json.gz', data_num=None, start_idx=0):
  review_texts = []
  ratings = []
  products = []
  pos=0
  neg=0
  root = '/content/drive/MyDrive/CS470_team_2in1'
  idx = 0
  with gzip.open(root+'/'+'dataset'+'/'+json_filename, "rb") as f:
    for line in f:
      idx+=1
      if start_idx>idx: continue
      
      info = json.loads(line)

      try:
        review_text = info["reviewText"]
        rating = int(info["overall"])
        product = info["asin"]
      except:
        continue
      if (rating>3 and pos<data_num/2):
        review_texts.append(review_text)
        ratings.append(rating)
        products.append(product)
        pos+=1
      elif (rating < 3 and neg < data_num/2):
        review_texts.append(review_text)
        ratings.append(rating)
        products.append(product)
        neg+=1

      if data_num and len(review_texts)==data_num: 
        print(f'index {idx}')
        break
      if len(review_texts)%1000==0: 
        print(f"processed {len(review_texts)} datas...")
        print(f'postive: {pos} negative: {neg}')

  print(f'collected {len(review_texts)} datas')
  return review_texts, ratings, products

In [ ]:
def generate_tokenized_dataset(review_texts, ratings, products):
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

  input_ids = []
  attention_masks = []

  for sent in review_texts:
      encoded_dict = tokenizer.encode_plus(
                          sent, 
                          add_special_tokens = True,
                          max_length = 512,
                          truncation=True,
                          pad_to_max_length = True,
                          return_attention_mask = True,
                          return_tensors = 'pt',
                    )
      
      input_ids.append(encoded_dict['input_ids'])
      attention_masks.append(encoded_dict['attention_mask'])

      if len(input_ids)%100000==0: print(f"tonkenized {len(input_ids)} datas...")

  dataset = np.array([input_ids, attention_masks, ratings, products])
  print("dataset is gernerated")
  return dataset


In [ ]:
review_test,rating_test,product_test = preprocess_amazon(data_num=100000,start_idx = 10000)
dataset_train = generate_tokenized_dataset(review_train, rating_train, product_train)
np.save("/content/drive/MyDrive/CS470_team_2in1/dataset"+"/"+"amazon_book_train_rev.npy", dataset_train)

review_test,rating_test,product_test = preprocess_amazon(data_num=20000,start_idx = 600000)
dataset_val = generate_tokenized_dataset(review_val, rating_val, product_val)
np.save("/content/drive/MyDrive/CS470_team_2in1/dataset"+"/"+"amazon_book_val_rev.npy", dataset_val)

review_test,rating_test,product_test = preprocess_amazon(data_num=20000,start_idx = 700000)
dataset_test = generate_tokenized_dataset(review_test, rating_test, product_test)
np.save("/content/drive/MyDrive/CS470_team_2in1/dataset"+"/"+"amazon_book_test_rev.npy", dataset_test)

In [ ]:
with open("/content/drive/MyDrive/CS470_team_2in1/dataset/"+'preprocessed_balanced_encoded.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)

    reviewText = []
    overall = []
    productId = []

    fields = ['index', 'sentence index', 'reviewText', 'overall', 'productID', 'positive rationale', 'negative rationale', 'positive rationale encoded', 'negative rationale encoded']
    
    for i, row in enumerate(csvreader):
        if(i==0): continue
        reviewText.append(row[2])
        overall.append(int(row[3]))
        productId.append(row[4])

dataset = generate_tokenized_dataset(reviewText, overall, productId)
np.save("/content/drive/MyDrive/CS470_team_2in1/dataset"+"/"+"rationale_preprocessed.npy", dataset)